In [1]:
a=0.002289058500149126*392
b=0.0023635258610685997*392
c=0.0024430012686149014*392

In [2]:
print(a,b,c)

0.8973109320584575 0.9265021375388911 0.9576564972970414


In [6]:
#you should always put the image at the second place 
import csv,os
def get_csv():
    class_path  =  'F:\\Ruiveen2\\Data\\Train_c2\\expert2\\'
    class_path3  = 'F:\\tq\\2019\\12\\23\\p2_unet_test2\\'
#     class_path3  = 'F:\\tq\\2019\\12\\6\\result\\case2\\deep_t2\\'
    blanket=set()
    #############case3
#     blanket.add(6)
#     blanket.add(21)
#     blanket.add(27)
# #################case1
#     blanket.add(8)
#     blanket.add(9)
#     blanket.add(2)
###############case2
    blanket.add(5)
    blanket.add(15)
    blanket.add(18)
###############case4
#     blanket.add(7)
#     blanket.add(29)
#     blanket.add(34)

#     if 2 in blanket:
#         print("fdsfdsfds")
    with open('eva_case2.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path):
            info=str.split(img_name,"_")
            
#             print(img_name)
#             break
#             print(info[0])
            if int(info[0]) in blanket:
                class_path2=class_path+img_name+"\\"
#                 print(class_path2)
#                 break
                for img_name2 in os.listdir(class_path2):
                    img_name3=info[1]+"_"+img_name2
                   # print(img_name3)
                    img_path2= class_path2+ img_name2
                    img_path = class_path3+"_"+info[1]+"_"+ img_name2
#                     print(img_path)
#                     print(img_path2)
#                     break
                    temp=[2]
                    temp.append(img_path)
                    temp.append(img_name3)
                    temp.append(img_path2)
                    temp.append("")
#                     print(temp)
#                     break
                    f_csv.writerow(temp)
                   # i=i+1
#                     if i%100==0:
#                         print(i)
#                     break
             #   break
get_csv()


In [7]:
# import csv,os
# class_path  = 'D:\\tq\\2019\\7\\10\\weekly_learning\\Train_c9\\'

# with open('my_test9.csv','w',newline='')as f:
#     f_csv = csv.writer(f)
#     i=0
#     for img_name in os.listdir(class_path):
#         if img_name[0]!='_':
#             continue
#         info=str.split(img_name[1:],".")
# #         print(info)
# #         print(img_name)
#       #  break
#         img_path = class_path + img_name
#         img_path2 = class_path+info[0]+"."+info[2]
        
#         temp=[info[1]]
#         temp.append(img_path)
#         temp.append(img_name)
#         temp.append(img_path2)
#         temp.append("")
#         print(temp)
#         break
#         f_csv.writerow(temp)
#         i=i+1
#         if i%100==0:
#             print(i)

In [2]:
import numpy as np
import torch
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
from copy import deepcopy
from matplotlib import cm
import imageio

def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z,W=[],[],[],[]
    O=[]
    i=1
    k=0
    for line in labels_list:
        info=str.split(line,",") # info[1]result； info[2]图片文件名
        if i==1:
            print("result: ",info[1])
            print("ground: ",info[3])
        try:
            a=imageio.imread(info[1])
        except FileNotFoundError:
            k+=1
            a=np.zeros((1024,512),np.uint8)
        X.append(a)
#         a=imageio.imread(info[1])
#         if a.shape[0]!=1024:
#             print(a.shape)
#             print(info[2])
#             O.append(info[2])
        W.append(imageio.imread(info[3]))
        Y.append(int(info[0]))
        Z.append(str(info[2]))
        i+=1
        if i%100==0:
            print(i)
       # break
   # print(O)
    print("not generate:",k)
    return list(zip(X,Y,Z,W))

In [7]:

data=load_data("eva_case2.csv")

result:  F:\tq\2019\12\23\p2_unet_test2\_17618_1.bmp
ground:  F:\Ruiveen2\Data\Train_c2\expert2\15_17618\1.bmp
100
200
300
400
500
600
not generate: 0


In [4]:
def oio():
    X,Y,_,W= zip(*data)
    X=np.array(X)
    W=np.array(W)
    cnt=0
    c=X.shape[1]
    d=X.shape[2]
#     print(c,d)
#     return
#     print(X.shape)
#     return
    TPVF=0
    DSC=0
    PPV=0
    MloU=0
    TPVFa=0
    TPVFb=0
    DSCa=0
    DSCb=0
    PPVa=0
    PPVb=0
    MloUa=0
    MloUb=0
    thred=127
    thred=0
    with open('my_nice2.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        for k in range(0,X.shape[0]):
            if np.sum(W[k])==0:
                continue
            cnt+=1
            TP=0
            G=0
            FP=0
            R=0
            img=X[k]
            img2=W[k]#Ground
            for i in range(c):
                for j in range(d):
                    if img2[i][j]>0:
                        G+=1
                        if img[i][j]>thred:
                            TP+=1
                            R+=1
                    elif img[i][j]>thred:
                        FP+=1
                        R+=1
            TPVFa+=TP
            TPVFb+=G
            DSCa+=TP*2
            DSCb+=R+G
            PPVa+=TP
            PPVb+=TP+FP
            MloUa+=TP
            MloUb+=FP+G
            if cnt%50==0:
                print(cnt)
            num1=G-TP
            num2=(R+G)-(TP*2)
            num3=FP#差距数值
            num4=(FP+G)-TP
            temp=[_[k],num1,num2,num3,num4,""]
            f_csv.writerow(temp)
            print(_[k],"                 ",num1,num2,num3,num4)
          #  break
    #         TPVF+=num1
    #         DSC+=num2
    #         PPV+=num3
    TPVF=TPVFa/TPVFb
    DSC=DSCa/DSCb
    PPV=PPVa/PPVb
    MloU=MloUa/MloUb
    print("TPVF= ",TPVF)
    print("DSC= ",DSC)
    print("PPV= ",PPV)
    print("MloU= ",MloU)

In [5]:
oio()# correct after segment case1 expert1 unet #0

17618_100.bmp                   254 13867 13613 13867
17618_101.bmp                   3216 15325 12109 15325
17618_102.bmp                   149 12336 12187 12336
17618_103.bmp                   522 12865 12343 12865
17618_104.bmp                   706 12146 11440 12146
17618_105.bmp                   6 10381 10375 10381
17618_106.bmp                   87 13174 13087 13174
17618_107.bmp                   231 15494 15263 15494
17618_108.bmp                   63 15046 14983 15046
17618_109.bmp                   197 15566 15369 15566
17618_110.bmp                   2150 20611 18461 20611
17618_111.bmp                   8 11598 11590 11598
17618_112.bmp                   22 11012 10990 11012
17618_113.bmp                   282 9815 9533 9815
17618_114.bmp                   79 10067 9988 10067
17618_115.bmp                   97 10861 10764 10861
17618_116.bmp                   69 11807 11738 11807
17618_117.bmp                   41 11355 11314 11355
17618_118.bmp                   48 9571 9

12665_80.bmp                   0 10748 10748 10748
12665_81.bmp                   37 12254 12217 12254
12665_82.bmp                   1 12759 12758 12759
12665_83.bmp                   0 11833 11833 11833
12665_84.bmp                   1 10376 10375 10376
12665_85.bmp                   2 12512 12510 12512
12665_86.bmp                   0 15638 15638 15638
12665_87.bmp                   1 13724 13723 13724
12665_88.bmp                   10 13762 13752 13762
12665_89.bmp                   3 12892 12889 12892
12665_9.bmp                   1020 1633 613 1633
12665_90.bmp                   15 12045 12030 12045
12665_91.bmp                   12 12793 12781 12793
12665_92.bmp                   0 13464 13464 13464
12665_93.bmp                   49 14899 14850 14899
12665_94.bmp                   10 14857 14847 14857
12665_95.bmp                   67 12646 12579 12646
12665_96.bmp                   58 12213 12155 12213
12665_97.bmp                   14 13828 13814 13828
12665_98.bmp            

9013_52.bmp                   3 4766 4763 4766
9013_53.bmp                   25 5268 5243 5268
9013_54.bmp                   0 4727 4727 4727
9013_55.bmp                   23 4259 4236 4259
9013_56.bmp                   4 4532 4528 4532
9013_57.bmp                   4 5065 5061 5065
9013_58.bmp                   2 3983 3981 3983
9013_59.bmp                   21 4684 4663 4684
9013_60.bmp                   4 4123 4119 4123
9013_61.bmp                   12 5325 5313 5325
9013_62.bmp                   0 5626 5626 5626
9013_63.bmp                   0 7356 7356 7356
9013_64.bmp                   60 4272 4212 4272
9013_65.bmp                   108 5827 5719 5827
9013_66.bmp                   94 5545 5451 5545
9013_67.bmp                   56 4971 4915 4971
9013_68.bmp                   21 3837 3816 3837
9013_69.bmp                   0 5629 5629 5629
9013_70.bmp                   8 5200 5192 5200
9013_71.bmp                   98 3700 3602 3700
9013_72.bmp                   21 4601 4580 4601
3

In [8]:


oio()#correct after segment case1 expert2 unet
#error exist in labels

17618_100.bmp                   231 14121 13890 14121
17618_101.bmp                   3171 16137 12966 16137
17618_102.bmp                   139 11937 11798 11937
17618_103.bmp                   510 12252 11742 12252
17618_104.bmp                   625 12048 11423 12048
17618_105.bmp                   5 10098 10093 10098
17618_106.bmp                   45 12004 11959 12004
17618_107.bmp                   66 14134 14068 14134
17618_108.bmp                   7 14762 14755 14762
17618_109.bmp                   129 14931 14802 14931
17618_110.bmp                   2053 20890 18837 20890
17618_111.bmp                   1 12101 12100 12101
17618_112.bmp                   0 11375 11375 11375
17618_113.bmp                   2 10361 10359 10361
17618_114.bmp                   14 11077 11063 11077
17618_115.bmp                   18 10893 10875 10893
17618_116.bmp                   14 10767 10753 10767
17618_117.bmp                   0 11031 11031 11031
17618_118.bmp                   24 10651 10

12665_82.bmp                   0 12864 12864 12864
12665_83.bmp                   0 12137 12137 12137
12665_84.bmp                   0 10377 10377 10377
12665_85.bmp                   0 12871 12871 12871
12665_86.bmp                   0 15394 15394 15394
12665_87.bmp                   0 14124 14124 14124
12665_88.bmp                   0 13583 13583 13583
12665_89.bmp                   0 13392 13392 13392
12665_9.bmp                   1175 1773 598 1773
12665_90.bmp                   0 12818 12818 12818
12665_91.bmp                   0 13226 13226 13226
12665_92.bmp                   0 15196 15196 15196
12665_93.bmp                   5 16514 16509 16514
12665_94.bmp                   0 15669 15669 15669
12665_95.bmp                   1 14881 14880 14881
12665_96.bmp                   1 14026 14025 14026
12665_97.bmp                   0 16779 16779 16779
12665_98.bmp                   0 15688 15688 15688
12665_99.bmp                   0 11304 11304 11304
14808_37.bmp                   0 

9013_50.bmp                   0 5740 5740 5740
9013_51.bmp                   36 4416 4380 4416
9013_52.bmp                   0 4523 4523 4523
9013_53.bmp                   0 3450 3450 3450
9013_54.bmp                   0 5179 5179 5179
9013_55.bmp                   0 4653 4653 4653
9013_56.bmp                   3 4544 4541 4544
9013_57.bmp                   3 4408 4405 4408
9013_58.bmp                   0 3701 3701 3701
9013_59.bmp                   0 4172 4172 4172
9013_60.bmp                   3 4402 4399 4402
9013_61.bmp                   2 4404 4402 4404
9013_62.bmp                   0 5486 5486 5486
9013_63.bmp                   1 6760 6759 6760
9013_64.bmp                   5 4003 3998 4003
9013_65.bmp                   6 5461 5455 5461
9013_66.bmp                   1 4634 4633 4634
9013_67.bmp                   64 3822 3758 3822
9013_68.bmp                   0 4285 4285 4285
350
9013_69.bmp                   1 5151 5150 5151
9013_70.bmp                   1 4838 4837 4838
9013_71

In [12]:
oio()#segment 23 case4
#correct label


14404_100.bmp                   1458 4184 2726 4184
14404_101.bmp                   1120 4890 3770 4890
14404_102.bmp                   1034 6007 4973 6007
14404_103.bmp                   725 6581 5856 6581
14404_104.bmp                   1119 6610 5491 6610
14404_105.bmp                   950 7022 6072 7022
14404_106.bmp                   1491 8461 6970 8461
14404_107.bmp                   843 7022 6179 7022
14404_108.bmp                   1593 5064 3471 5064
14404_109.bmp                   1672 5553 3881 5553
14404_110.bmp                   1048 3452 2404 3452
14404_111.bmp                   1126 4394 3268 4394
14404_112.bmp                   2623 5238 2615 5238
14404_113.bmp                   1395 6105 4710 6105
14404_114.bmp                   601 6856 6255 6856
14404_115.bmp                   348 4297 3949 4297
14404_116.bmp                   466 2672 2206 2672
14404_117.bmp                   620 2701 2081 2701
14404_118.bmp                   707 6176 5469 6176
14404_119.bmp       

In [3]:
import csv,os
class_path  = 'D:\\`atq\\2019\\7\\10\\weekly_learning\\Train_c12\\'

with open('my_test12.csv','w',newline='')as f:
    f_csv = csv.writer(f)
    i=0
    for img_name in os.listdir(class_path):
        if img_name[0]!='_':
            continue
        info=str.split(img_name[1:],".")
#         print(info)
#         print(img_name[1:])
#         break
        img_path = class_path + img_name[1:]
        img_path2 = class_path+img_name
        
        temp=[info[1]]
        temp.append(img_path)
        temp.append(img_name[1:])
        temp.append(img_path2)
        temp.append("")
        f_csv.writerow(temp)
        i=i+1
        if i%100==0:
            print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
